In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 1. Load Data
data = "properties.csv"
df = pd.read_csv(data, encoding='utf-8')

print("Original Data Shape:", df.shape)
print(df.info())

/var/folders/_1/w3rmdmrx14qdr85c4l_crhq80000gn/T/ipykernel_9746/4269065727.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data, encoding='utf-8')


Original Data Shape: (1262519, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262519 entries, 0 to 1262518
Data columns (total 16 columns):
 #   Column                                          Non-Null Count    Dtype  
---  ------                                          --------------    -----  
 0   Building unit type                              1262519 non-null  object 
 1   Building Location                               1262519 non-null  object 
 2   Building Estate                                 1262519 non-null  object 
 3   District                                        1262519 non-null  object 
 4   Longitude                                       1262519 non-null  float64
 5   Latitude                                        1262519 non-null  float64
 6   Building Floor                                  1262190 non-null  object 
 7   Building Unit                                   1260570 non-null  object 
 8   Building Net Area                               1262519 n

In [3]:
# 2. Data Cleaning & Preprocessing
# Based on version_1.ipynb logic

# Drop useless columns
columns_to_remove = ['Price per gross area','Building Gross Area','Building unit type','Longitude','Latitude','Building Unit','% change from previous transaction (same unit)', 'date of previous transaction']
df = df.drop(columns=columns_to_remove)

# Convert numeric columns (removing commas)
def convert_to_numeric(series):
    if series.dtype == 'object':
        return pd.to_numeric(series.str.replace(',', ''), errors='coerce')
    return series

# Convert relevant columns
df['Building Net Area'] = convert_to_numeric(df['Building Net Area'])
df['Price per net area'] = convert_to_numeric(df['Price per net area'])
# Building Price hkd is usually int, but apply just in case or leave if already int
# df['Building Price hkd'] is int64 in version_1, so no change needed.

# Convert Transaction date to datetime
df['Transaction date'] = pd.to_datetime(df['Transaction date'])

# Handle Building Floor (Optional: convert to numeric if possible, coerce errors to NaN)
# Keeping it consistent with numeric conversion
# Note: Some floors might be 'G', 'M', etc. This will turn them to NaN. 
# Assuming this is acceptable for numeric modeling or further cleaning.
df['Building Floor'] = convert_to_numeric(df['Building Floor'])

print("Data Shape after cleaning:", df.shape)
print(df.info())

Data Shape after cleaning: (1262519, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262519 entries, 0 to 1262518
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   Building Location   1262519 non-null  object        
 1   Building Estate     1262519 non-null  object        
 2   District            1262519 non-null  object        
 3   Building Floor      898371 non-null   float64       
 4   Building Net Area   867970 non-null   float64       
 5   Building Price hkd  1262519 non-null  int64         
 6   Price per net area  867970 non-null   float64       
 7   Transaction date    1262519 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 77.1+ MB
None


In [4]:
# 3. Define Train/Test Split Masks
# Training: Data before 2017
# Testing: Data in 2017 and 2018

train_mask = df['Transaction date'] < '2017-01-01'
test_mask = (df['Transaction date'] >= '2017-01-01') & (df['Transaction date'] <= '2018-12-31')

print(f"Training samples (< 2017): {train_mask.sum():,}")
print(f"Testing samples (2017-2018): {test_mask.sum():,}")
print(f"Other samples: {(~train_mask & ~test_mask).sum():,}")

Training samples (< 2017): 1,172,842
Testing samples (2017-2018): 75,414
Other samples: 14,263


In [5]:
# 4. Feature Engineering: Property Type (One-Hot Encoding)
# IMPORTANT: Calculate Statistics (Q1, Q3) using ONLY Training Data to avoid leakage

train_prices = df.loc[train_mask, 'Price per net area'].dropna()
Q1 = train_prices.quantile(0.25)
Q3 = train_prices.quantile(0.75)

print(f"Training Data Q1: {Q1}")
print(f"Training Data Q3: {Q3}")

def classify_property_type(price):
    if pd.isna(price):
        return np.nan
    elif price < Q1:
        return 'Economy'
    elif price < Q3:
        return 'Standard'
    else:
        return 'Premium'

# Apply to ALL data
df['Property_Type'] = df['Price per net area'].apply(classify_property_type)

# Create One-Hot Encoding
df['Property_Economy'] = (df['Property_Type'] == 'Economy').astype(int)
df['Property_Standard'] = (df['Property_Type'] == 'Standard').astype(int)
df['Property_Premium'] = (df['Property_Type'] == 'Premium').astype(int)

# Handle NaNs (set all one-hot to 0 if type is NaN)
df.loc[df['Property_Type'].isna(), ['Property_Economy', 'Property_Standard', 'Property_Premium']] = 0

print("Property Type Feature Created.")

Training Data Q1: 3664.0
Training Data Q3: 7543.0
Property Type Feature Created.


In [6]:
# 5. Feature Engineering: District Target Encoding
# IMPORTANT: Calculate Mean Price using ONLY Training Data to avoid leakage

# Calculate mean price per district on Training Data
district_means = df.loc[train_mask].groupby('District')['Building Price hkd'].mean()

# Map to ALL data
df['District_Target_Encoded'] = df['District'].map(district_means)

# Handle missing values (Districts in Test but not in Train)
# Fill with global mean of Training Data
global_mean_price = df.loc[train_mask, 'Building Price hkd'].mean()
df['District_Target_Encoded'] = df['District_Target_Encoded'].fillna(global_mean_price)

print("District Target Encoded Feature Created.")
print(f"Global Mean Price (Train): {global_mean_price:,.2f}")

District Target Encoded Feature Created.
Global Mean Price (Train): 401.47


In [7]:
# 6. Final Dataset Preparation
# Drop intermediate columns if needed, or just select X and y directly.

# Clean up NaNs in features used for model (Optional but recommended)
# For this version, we will drop rows with NaN in essential columns or fill them.
# Let's check NaNs first
print("Missing Values before drop/fill:")
print(df[["Building Floor", "Building Net Area", "District_Target_Encoded"]].isnull().sum())

# Drop rows where essential features are NaN (e.g., Net Area, Floor)
# Note: 'Price per net area' might be NaN, but we are using 'Building Price hkd' as target.
# 'Building Floor' might be NaN after conversion if it was 'G' etc.

df_clean = df.dropna(subset=["Building Floor", "Building Net Area", "Building Price hkd"])

# Re-apply masks on clean data
train_mask_clean = df_clean['Transaction date'] < '2017-01-01'
test_mask_clean = (df_clean['Transaction date'] >= '2017-01-01') & (df_clean['Transaction date'] <= '2018-12-31')

# 7. Split into X and y
feature_cols = [
    "Building Floor", 
    "Building Net Area", 
    "Transaction date",
    "Property_Economy", 
    "Property_Standard", 
    "Property_Premium", 
    "District_Target_Encoded"
]
target_col = "Building Price hkd"

X_train = df_clean.loc[train_mask_clean, feature_cols]
y_train = df_clean.loc[train_mask_clean, target_col]

X_test = df_clean.loc[test_mask_clean, feature_cols]
y_test = df_clean.loc[test_mask_clean, target_col]

print("="*50)
print("Training Set:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("="*50)
print("Testing Set (2017-2018):")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("="*50)

# Display head of X_train
print(X_train.head())

Missing Values before drop/fill:
Building Floor             364148
Building Net Area          394549
District_Target_Encoded         0
dtype: int64
Training Set:
X_train shape: (589364, 7)
y_train shape: (589364,)
Testing Set (2017-2018):
X_test shape: (39325, 7)
y_test shape: (39325,)
   Building Floor  Building Net Area Transaction date  Property_Economy  \
0            26.0              787.0       2016-08-25                 0   
1            26.0              787.0       1996-11-26                 0   
2            26.0              624.0       2016-11-28                 1   
3            26.0              624.0       2011-08-22                 0   
4            26.0              624.0       1997-04-14                 0   

   Property_Standard  Property_Premium  District_Target_Encoded  
0                  0                 1               398.185188  
1                  0                 1               398.185188  
2                  0                 0               398.185188 